# Identifying PII in Student Essays
## Project Summary
The Kaggle Competition we are participating in is the [PII Data Detection hosted by The Learning Agency Lab](https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/overview). The goal of this competition is to develop a model that detects sensitive personally identifiable information (PII) in student writing. This is necessary to screen and clean educational data so that when released to the public for analysis and archival, the students' risk are mitigated.

## Cloning Repo
Because one of the files is larger than 100MiB, the file could not be uploaded directly to the github repo. The solution found was using git large file system to hold the file and upload the git lfs pointer file in the place of the json.

Git Bash Code:
```
# install git lfs
git lfs install

# start file tracking for git lfs in the repo
git lfs track "*.json"

# stage/commit/push training json
git add train.json
git commit -m "add train.json"
git push
```
After cloning the repo locally, it clones the git lfs pointer file not the data file.

Git Bash Code:
```
# pull file from git lfs system into local repo using any pointer files
git lfs pull
```


## External Data Sources

* [Persuade PII Dataset](https://www.kaggle.com/datasets/thedrcat/persuade-pii-dataset?rvi=1)
  * Essays from Persuade corpus, modified with synthetic PII data and corresponding labels. It was filtered for essays that contain tokens that are relevant to competition.

* [PII | External Dataset](https://www.kaggle.com/datasets/alejopaullier/pii-external-dataset?rvi=1)
  * This is an LLM-generated external dataset that contains generated texts with their corresponding annotated labels in the required competition format.

* [NEW DATASET PII Data Detection](https://www.kaggle.com/datasets/cristaliss/new-dataset-pii-data-detection?rvi=1)
  * This dataset is a modified version of the official training which have the following changes: Revamped Labels, Token Transformation, and Token indexing

* [PII Detection Dataset (GPT)](https://www.kaggle.com/datasets/pjmathematician/pii-detection-dataset-gpt)
  * Personal data was created using python Faker package, which was then fed into the LLM to write an essay on. Overall, it contains 2000 gpt - generated essays and corresponding competition entities used in the essay.

* [AI4privacy-PII](https://www.kaggle.com/datasets/verracodeguacas/ai4privacy-pii)
  * The dataset is crafted using proprietary algorithms, ensuring the creation of synthetic data that avoids privacy violations. The data is meticulously curated with human-in-the-loop validation, ensuring both relevance and quality. It serves a crucial role in addressing the growing concerns around personal data security in AI applications.



## Python Libraries


In [3]:
try:
    import pandas as pd
    import numpy as np
    import spacy as sp
    import re
    from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
    import json
    from pathlib import Path
    from datasets import Dataset
    import evaluate
except DeprecationWarning:
    None

ModuleNotFoundError: No module named 'spacy'

## Loading Datasets


### Official training data

Only load into notebook after pulling from git LFS (see above)

In [4]:
df_train = pd.read_json("../Datasets/Official/train.json")
df_train

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


### Official testing data


In [5]:
df_test = pd.read_json("../Datasets/Official/test.json")
df_test

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."
5,86,Cheese Startup - Learning Launch ​by Eladio Am...,"[Cheese, Startup, -, Learning, Launch, ​by, El...","[True, True, True, True, True, True, True, Fal..."
6,93,Silvia Villalobos\n\nChallenge:\n\nThere is a ...,"[Silvia, Villalobos, \n\n, Challenge, :, \n\n,...","[True, False, False, False, False, False, True..."
7,104,Storytelling The Path to Innovation\n\nDr Sak...,"[Storytelling, , The, Path, to, Innovation, \...","[True, False, True, True, True, False, False, ..."
8,112,Reflection – Learning Launch\n\nFrancisco Ferr...,"[Reflection, –, Learning, Launch, \n\n, Franci...","[True, True, True, False, False, True, False, ..."
9,123,Gandhi Institute of Technology and Management ...,"[Gandhi, Institute, of, Technology, and, Manag...","[True, True, True, True, True, True, False, Tr..."


## Cleaning
To have some uniform input, each source dataframe needs to have a list of tokens from of the source text located in each row.

### Official training dataset
Verify that there are no rows with any null values

In [6]:
# df_train[df_train.isnull().any(axis = 1)]

### Official test dataset
Verify that there are no rows with any null values

In [7]:
df_test[df_test.isnull().any(axis = 1)]

,document,full_text,tokens,trailing_whitespace


# Preparing Model Inputs

### Loading and Cleaning Datasets

In [8]:
import pandas as pd
import numpy as np
import spacy as sp
import re
import os
import transformers
import torch
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


import torch.nn as nn
from torch import cuda
# from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification, AutoTokenizer



import warnings
warnings.filterwarnings('ignore')
torch.cuda.empty_cache()

ModuleNotFoundError: No module named 'spacy'

### Config

This section does initial configuration, loading our the bert pretained model that we use, as well as setting up hyper parameters like `EPOCHS` and `LEARNING_RATE`

In [ ]:
class Config():
    def __init__(self, platform, model_name, pretrained_model_name):
        # platform = 'Kaggle'# 
        if platform == 'Kaggle':
            pretrained_model = '../input/huggingface-bert/' + pretrained_model_name + '/'
            train_path = 'path_TBD/train.json'
            # test_path = ''
            model_path = 'path_TBD/' + model_name
        elif platform == 'local':
            model_path = '../models/bert_models/' + model_name
        
        self.config = {
            'MAX_LEN': 128,
            'TRAIN_BATCH_SIZE': 4,
            'VALID_BATCH_SIZE': 2,
            'EPOCHS': 1,
            'LEARNING_RATE':1e-5,
            'MAX_GRAD_NORM': 10,
            'device': 'cuda' if cuda.is_available() else 'cpu',
            # 'device': 'cpu',
            'model_path': model_path,
            'pretrained_model': pretrained_model_name,
            'tokenizer': BertTokenizerFast.from_pretrained(pretrained_model_name),
            'threshold': 0.9
        }

In [ ]:
platform = 'local'
pretrainend_model_name = 'bert-base-cased'
model_num = 1
model_name = 'model' + str(model_num) + '-' + pretrainend_model_name +'.bin'

config = Config(platform,model_name, pretrainend_model_name).config

Requires:
```git lfs track "*.safetensors"```
to clone model from github

If using IDE, Run 
```
python -m spacy download en_core_web_sm
```
in the bash to install the english spacy pipline

## Official Datasets

### Data and Exploration

In [ ]:
df = pd.read_json("../Datasets/Official/train.json")
df['labels']

In [ ]:
# df[df.isnull().any(axis = 1)]

## Hypothesis testing

### Proportion of PII as a percentage of the text

H0: The proportion of PII in each text is zero.​

Ha: The proportion of PII in each text is not zero.

In [ ]:
import numpy as np

labels = df['labels']

non_o_values = df.apply(lambda line: sum(1 for x in line['labels'] if x != 'O')/ len(line['labels']), axis = 1)

proportions_array = np.array(non_o_values)

plt.hist(proportions_array, bins=20, color='blue', edgecolor='black', alpha=0.7)
plt.xlabel('Proportion of PII Values in Text')
plt.ylabel('Frequency (Log Scaled)')
plt.yscale('log')
plt.title('Distribution of PII Values in Text')
plt.show()

Since the p-value is below alpha (0.01) we reject the null hypothesis.

In [ ]:
import scipy.stats as stats
t_statistic, p_value = stats.ttest_1samp(proportions_array, 0)

print("t-statistic:", t_statistic)
print("p-value:", p_value)

### Proprtion of each type of PII

H0: All PII have equal likelihood of appearing.​

Ha: Name PII are more likely to appear.​

In [ ]:
from collections import Counter
c = Counter()
df.apply(lambda line: c.update(line.labels), axis = 1)
c_pii = c.most_common()[1:]
c_key, c_val = zip(*c_pii)
plt.barh(c_key, c_val)
# plt.ylabel('PII Labels', horizontalalignment='left')
plt.title("Occurrences of PII Type in Text")
# plt.subplots_adjust(left=0.2)
# plt.gca().get_yticklabels() # This gets the current y-axis tick labels
pos = plt.gca().get_position()
plt.gca().set_position([pos.x0 + 0.9, pos.y0 + 0.9, pos.width * 0.9, pos.height * 0.9])
[tl.set_horizontalalignment('left') for tl in plt.gca().get_yticklabels()]
plt.show()

### Positional distribution of most popular PII

H0: PII is evenly distributed across the length of an essay.​​

Ha: PII is more likely to appear in the beginning of the essay.​

In [ ]:
# Creates a dictionary for each position in the text (beginning, middle, and end)
positions = {
    'beginning': lambda x: x[:int(len(x)/3)],
    'middle': lambda x: x[int(len(x)/3):int(-len(x)/3)],
    'end': lambda x: x[int(-len(x)/3):-1]
}

label_counts = {pos: {} for pos in positions}
for arr in df['labels']:
    for pos, func in positions.items():
            arr = func(arr)
            for label in arr:
                if label in label_counts[pos]:
                        label_counts[pos][label] += 1
                else:
                        label_counts[pos][label] = 1


beginning_labels = label_counts['beginning']
middle_labels = label_counts['middle']
end_labels = label_counts['end']

# B-NAME_STUDENT
# I-NAME_STUDENT
# B-URL_PERSONAL

B_name = [beginning_labels['B-NAME_STUDENT'],
        middle_labels['B-NAME_STUDENT'],
        end_labels['B-NAME_STUDENT']]

I_name = [beginning_labels['I-NAME_STUDENT'],
        middle_labels['I-NAME_STUDENT'],
        end_labels['I-NAME_STUDENT']]

B_url = [beginning_labels['B-URL_PERSONAL'],
        middle_labels['B-URL_PERSONAL'],
        end_labels['B-URL_PERSONAL']]

locations = ['Beginning', 'Middle', 'End']

fig, ax = plt.subplots(1,3, figsize=(15, 5))
fig.suptitle('Frequency of most popular labels at different positions in the text')
ax[0].set_ylabel('Frequency')
ax[0].bar(locations, B_name)
ax[0].set_title('B-Name')
ax[1].bar(locations, I_name)
ax[1].set_title('I-Name')
ax[2].bar(locations, B_url)
ax[2].set_title('B-URL')

In [ ]:
c_pii

In [ ]:
labels_to_ids = {k: v for v, k in enumerate(c.keys())}
ids_to_labels = {v: k for v, k in enumerate(c.keys())}
labels_to_ids

## Preprocessing

First we scoured the data to find the usuable text with our pretrained model with Regex.

In [ ]:
pattern = re.compile('\xa0|\uf0b7|\u200b')
df.loc[:,'full_text'] = df.loc[:,'full_text'].replace(pattern, ' ')
df.loc[:,'tokens'] = df.loc[:,'tokens'].apply(lambda line: [tok for tok in line if not re.search(pattern,tok)])

df_usable = df.iloc[df[~(df.tokens.apply(len) != df.labels.apply(len))].index]
1-(len(df_usable))/len(df.document)

In [ ]:
df_usable

We created a function to tokenize and label all of the lines in the dataset.

In [ ]:
def make_smaller_inputs(dataframe):
    df_out = pd.DataFrame(columns = ['tokens','labels','document','document_location'])
    idx_df = 0
    max_len = config['MAX_LEN']
    
    for _,line in dataframe.iterrows():
        location_counter = 0
        tokens = line.tokens
        labels = line.labels
        document = line.document
        items = range(0,len(tokens),max_len)
        
        for i in items:
            df_out.at[idx_df,'tokens'] = tokens[i:i+max_len]
            df_out.at[idx_df,'labels'] = labels[i:i+max_len]
            df_out.at[idx_df,'document'] = document
            df_out.at[idx_df,'document_location'] = location_counter
            location_counter += 1
            idx_df += 1
        
    return df_out

In [ ]:
df_model_input = make_smaller_inputs(df_usable)
df_model_input.head(10)

In [ ]:
len(df_model_input.index)

### Creating the training and testing split

In [ ]:
df_train, df_test = train_test_split(df_model_input, test_size=0.3)

### Formatting split

In [ ]:
df_train.reset_index(drop = True, inplace=True)
df_train

In [ ]:
df_test.reset_index(drop = True, inplace=True)
df_test

# Model


## Iteration 1

adapted from 
https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb#scrollTo=Eh3ckSO0YMZW

### Tokenizer and Label allignment

In [ ]:
# class dataset(Dataset):
#     def __init__(self, dataframe, tokenizer, max_len):
#         self.len = len(dataframe)
#         self.data = dataframe
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __getitem__(self, index):
#         # step 1: get the sentence and word labels 
#         tokens = self.data.tokens[index]
#         word_labels = self.data.labels[index]

#         # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
#         # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
#         encoding = self.tokenizer(tokens,
#                                   is_split_into_words=True,
#                                   return_offsets_mapping=True,
#                                   padding='max_length',
#                                   truncation=True,
#                                   max_length=self.max_len)

#         # step 3: create token labels only for first word pieces of each tokenized word
#         labels = [ labels_to_ids[label] for label in word_labels]
#         # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
#         # create an empty array of -100 of length max_length
#         encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

#         # set only labels whose first offset position is 0 and the second is not 0
#         i = 0
#         for idx, mapping in enumerate(encoding["offset_mapping"]):
#             if mapping[0] == 0 and mapping[1] != 0:
#                 # overwrite label
#                 encoded_labels[idx] = labels[i]
#                 i += 1

#         # step 4: turn everything into PyTorch tensors
#         item = {key: torch.as_tensor(val) for key, val in encoding.items()}
#         item['labels'] = torch.as_tensor(encoded_labels)

#         return item

#     def __len__(self):
#         return self.len
# training_set = dataset(df_train, config['tokenizer'], config['MAX_LEN'])
# testing_set = dataset(df_test, config['tokenizer'], config['MAX_LEN'])
# training_set[2]["input_ids"].unsqueeze(0)
# for token, label in zip(config['tokenizer'].convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
#     print('{0:10}  {1}'.format(token, label))

Setting training and testing parameters

In [ ]:
# train_params = {'batch_size': config["TRAIN_BATCH_SIZE"],
#                 'shuffle': True,
#                 'num_workers': 0
#                 }

# test_params = {'batch_size': config["VALID_BATCH_SIZE"],
#                'shuffle': True,
#                'num_workers': 0
#                }

# training_loader = DataLoader(training_set, **train_params)
# testing_loader = DataLoader(testing_set, **test_params)
# model = BertForTokenClassification.from_pretrained(config['pretrained_model'], num_labels=len(labels_to_ids))
# model.to(config['device'])
# optimizer = torch.optim.Adam(params=model.parameters(), lr=config['LEARNING_RATE'])

### Training

In [ ]:
### Training
# # Defining the training function on the 80% of the dataset for tuning the bert model
# def train(epoch):
#     tr_loss, tr_accuracy = 0, 0
#     nb_tr_examples, nb_tr_steps = 0, 0
#     tr_preds, tr_labels = [], []
#     # put model in training mode
#     model.train()
# 
#     for idx, batch in enumerate(training_loader):
# 
#         ids = batch['input_ids'].to(config['device'], dtype = torch.long)
#         mask = batch['attention_mask'].to(config['device'], dtype = torch.long)
#         labels = batch['labels'].to(config['device'], dtype = torch.long)
# 
#         outputs = model(input_ids=ids.long(), attention_mask=mask.long(), labels=labels.long())
#         loss = outputs[0]
#         tr_logits = outputs[1]
#         tr_loss += loss.item()
# 
#         nb_tr_steps += 1
#         nb_tr_examples += labels.size(0)
# 
#         if idx % 100==0:
#             loss_step = tr_loss/nb_tr_steps
#             print(f"Training loss per 100 training steps: {loss_step}")
# 
#         # compute training accuracy
#         flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
#         active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
#         flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
#         
#         # only compute accuracy at active labels
#         active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
#         #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
# 
#         labels = torch.masked_select(flattened_targets, active_accuracy)
#         predictions = torch.masked_select(flattened_predictions, active_accuracy)
# 
#         tr_labels.extend(labels)
#         tr_preds.extend(predictions)
# 
#         tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
#         tr_accuracy += tmp_tr_accuracy
# 
#         # gradient clipping
#         torch.nn.utils.clip_grad_norm_(
#             parameters=model.parameters(), max_norm=config['MAX_GRAD_NORM']
#         )
# 
#         # backward pass
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
# 
#     epoch_loss = tr_loss / nb_tr_steps
#     tr_accuracy = tr_accuracy / nb_tr_steps
#     print(f"Training loss epoch: {epoch_loss}")
#     print(f"Training accuracy epoch: {tr_accuracy}")
# run = False
# if run:
#     for epoch in range(config['EPOCHS']):
#         print(f"Training epoch: {epoch + 1}")
#         train(epoch)
#     run = False

Training epoch: 1
Training loss epoch: 0.007183262327166176
Training accuracy epoch: 0.9988009906964034

Execution Time: 2h 33m 45s
Batch Size: 2
LR: 1e-5
Optimizer: Adam

### Save Model

In [ ]:
# import os

# directory = config['model_path']

# if not os.path.exists(directory):
#     os.makedirs(directory)

# # save vocabulary of the tokenizer
# config['tokenizer'].save_vocabulary(directory)
# # save the model weights and its configuration file
# save_model = True
# if save_model:  
#     model.save_pretrained(directory)
#     print('All files saved')

### Model Validation

In [ ]:
# model_1 = BertForTokenClassification.from_pretrained(config['model_path'], num_labels=len(labels_to_ids))
# model_1.to(config['device'])
# # def valid(model, testing_loader):
#     # put model in evaluation mode
#     model.eval()
    
#     eval_loss, eval_accuracy = 0, 0
#     nb_eval_examples, nb_eval_steps = 0, 0
#     eval_preds, eval_labels = [], []
    
#     with torch.no_grad():
#         for idx, batch in enumerate(testing_loader):
            
#             ids = batch['input_ids'].to(config['device'], dtype = torch.long)
#             mask = batch['attention_mask'].to(config['device'], dtype = torch.long)
#             labels = batch['labels'].to(config['device'], dtype = torch.long)

#             outputs = model(input_ids=ids.long(), attention_mask=mask.long(), labels=labels.long())
            
#             loss = outputs[0]
#             eval_logits = outputs[1]
#             eval_loss += loss.item()
            
#             nb_eval_steps += 1
#             nb_eval_examples += labels.size(0)
        
#             if idx % 100==0:
#                 loss_step = eval_loss/nb_eval_steps
#                 print(f"Validation loss per 100 evaluation steps: {loss_step}")
            
#             # compute evaluation accuracy
#             flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
#             active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
#             flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
#             # only compute accuracy at active labels
#             active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
#             labels = torch.masked_select(flattened_targets, active_accuracy)
#             predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
#             eval_labels.extend(labels)
#             eval_preds.extend(predictions)
            
#             tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
#             eval_accuracy += tmp_eval_accuracy

#     labels = [ids_to_labels[id.item()] for id in eval_labels]
#     predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
#     eval_loss = eval_loss / nb_eval_steps
#     eval_accuracy = eval_accuracy / nb_eval_steps
#     print(f"Validation Loss: {eval_loss}")
#     print(f"Validation Accuracy: {eval_accuracy}")

#     return labels, predictions

# labels, predictions = valid(model_1, testing_loader)

Validation Loss: 0.0025191350434306515
Validation Accuracy: 0.9994422838479019
Execution Time: 08m 07.238s

In [ ]:
# print(classification_report(labels, predictions))

About 50% for getting student names correct, 0% for the others

## Iteration 2

Updating the tokenizer to properly create context between the words and the PII labels

In [ ]:
# def tokenize(df, tokenizer):
#     text = []
#     token_map = []
#     
#     idx = 0
#     
#     for t, ws in zip(df["tokens"], df["trailing_whitespace"]):
#         
#         text.append(t)
#         token_map.extend([idx]*len(t))
#         if ws:
#             text.append(" ")
#             token_map.append(-1)
#             
#         idx += 1
#         
#         
#     tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=False)
#     
#         
#     return {
#         **tokenized,
#         "token_map": token_map,
#     }

### LOOK AT ME FROM HERE

### Tokenizer and Label allignment

In [ ]:
def tokenize(line, tokenizer):
    # step 1: get the sentence and word labels 
    tokens = line.tokens
    word_labels = line.labels
    document = line.document
    location = line.document_location
    # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
    # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
    encoding = tokenizer(tokens,
                              is_split_into_words=True,
                              return_offsets_mapping=True,
                              padding='max_length',
                              max_length=2 * config['MAX_LEN']
                              )
    temp_list = [0 for _ in range(2 * config['MAX_LEN'] - len(word_labels))]
    labels = [labels_to_ids[label] for label in word_labels] + temp_list
    
    encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * 0
    # WORKS
    i = 0
    for idx, mapping in enumerate(encoding["offset_mapping"]):        
        if mapping[0] == 0:
            i += 1
    # step 4: turn everything into PyTorch tensors
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
            
    item['labels'] = torch.as_tensor(encoded_labels)
    # print(items) TODO
    return {'item': item, 'document': document, 'location': location}


Create the mappings for the tokenized text, calling the tokenizer function above. This will store all of the values in separate lists that we will be able to call for the output function.

In [ ]:
# df_train.iloc[0,:]

In [ ]:
# with open("cleaned_train.json", "w") as data_file:
#     data_file.write(df_train.to_json())
#     data_file.close()

# with open("cleaned_train.json", "r") as f:
#     data = json.load(f)

# with open("../datasets/Official/train.json", "r") as f:
#   data = json.load(f)

# print(data)

# ds = Dataset.from_dict({
#     # "full_text": [x["full_text"] for x in data],
#     "labels": data["labels"],
#     "tokens": data["tokens"],
#     # "trailing_whitespace": [x["trailing_whitespace"] for x in data],
# })


# tokenizer = BertTokenizerFast.from_pretrained(config['model_path'])

In [ ]:
train_dataset_temp = df_train.apply(lambda line: tokenize(line, config['tokenizer']), axis = 1).to_list()
train_dataset = pd.DataFrame.from_dict(train_dataset_temp, orient = 'columns')
train_dataset.head()

In [ ]:
predict_dataset_temp = df_test.apply(lambda line: tokenize(line, config['tokenizer']), axis = 1).to_list()
predict_dataset = pd.DataFrame.from_dict(predict_dataset_temp, orient = 'columns')
predict_dataset.head()

In [ ]:
eval_dataset = predict_dataset

Create the new training class, with the model creation leveraging AutoModelForTokenClassification

In [ ]:
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # # Remove ignored index (special tokens)
    # true_predictions = [
    #     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    #     for prediction, label in zip(predictions, labels)
    # ]
    # true_labels = [
    #     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    #     for prediction, label in zip(predictions, labels)
    # ]

    results = metric.compute(predictions=predictions, references=labels)
    if data_args.return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [ ]:
model = BertForTokenClassification.from_pretrained(config['pretrained_model'])
data_collator = DataCollatorForTokenClassification(config['tokenizer'])
# optimizer = torch.optim.Adam(params=model.parameters(), lr=config['LEARNING_RATE'])

### Training

In [ ]:
torch.cuda.is_available()

In [ ]:
training_args = TrainingArguments(
    output_dir= config['model_path'],
    overwrite_output_dir = True,
    do_train = True,
    do_predict = False,
    do_eval = True,
    per_device_eval_batch_size=1, 
    report_to="none",
    num_train_epochs = config['EPOCHS'],
    learning_rate = config['LEARNING_RATE'],
    save_strategy = 'epoch',
    disable_tqdm= True,
    no_cude = False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset.item,
    eval_dataset = train_dataset.item,
    # predict_dataset = predict_dataset.item,
    # tokenizer=config['tokenizer'],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

Next, we call the actual training function that will supply the model created above with the information that it needs to create context between text, and the labels. It will work out which values match the closes and assign the proper label to it, based on how well that label matches the value.

https://github.com/huggingface/transformers/blob/main/examples/pytorch/token-classification/run_ner.py#L546

In [ ]:
# Training
if training_args.do_train:
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.save_model()  # Saves the tokenizer too for easy upload

    # max_train_samples = (
    #     data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    # )
    metrics["train_samples"] = len(train_dataset)

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    # trainer.save_state()
model = BertForTokenClassification.from_pretrained(config['model_path'], num_labels=len(labels_to_ids))

In [ ]:
# # Evaluation
# if training_args.do_eval:
#     logger.info("*** Evaluate ***")
# 
#     metrics = trainer.evaluate()
# 
#  #   max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
#     metrics["eval_samples"] = len(eval_dataset)
# 
#     trainer.log_metrics("eval", metrics)
#     trainer.save_metrics("eval", metrics)

### Prediction

In [ ]:
# Predict
if training_args.do_predict:
    # logger.info("*** Predict ***")

    predictions, labels, metrics = trainer.predict(predict_dataset, metric_key_prefix="predict")
    # predictions = np.argmax(predictions, axis=2)

    # # Remove ignored index (special tokens)
    # true_predictions = [
    #     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    #     for prediction, label in zip(predictions, labels)
    # ]

    trainer.log_metrics("predict", metrics)
    trainer.save_metrics("predict", metrics)

    # Save predictions
    # output_predictions_file = os.path.join(training_args.output_dir, "predictions.txt")
    # if trainer.is_world_process_zero():
    #     with open(output_predictions_file, "w") as writer:
    #         for prediction in true_predictions:
    #             writer.write(" ".join(prediction) + "\n")


### TO HERE

Create the lists, pulling the context of the list. This will look at the token_map and place the proper label in it's own list, which is associated with the index of the corresponding item in the other list.

It ensures that we are ignoring the o labels so that we don't report those in our created file.

In [9]:
# predictions = trainer.predict(ds).predictions
preds_temp = np.argmax(predictions, axis=2)
# pred_softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis = 2).reshape(predictions.shape[0],predictions.shape[1],1)
id2label = ids_to_labels
# preds_without_O = pred_softmax[:,:,:12].argmax(-1)
# O_preds = pred_softmax[:,:,12]
preds_temp

NameError: name 'predictions' is not defined

In [10]:
# preds_final = np.where(O_preds < config['threshold'], preds_without_O , preds)
# preds_final = predictions.argmax(-1)
preds_final

NameError: name 'preds_final' is not defined

In [11]:
pairs = set()
document, token, label, token_str = [], [], [], []
for p, token_map, offsets, tokens, doc in zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"]):

    for token_pred, (start_idx, end_idx) in zip(p, offsets):
        label_pred = id2label[str(token_pred)]

        if start_idx + end_idx == 0: 
            continue

        if token_map[start_idx] == -1:
            start_idx += 1

        # ignore "\n\n"
        while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
            start_idx += 1

        if start_idx >= len(token_map): 
            break

        token_id = token_map[start_idx]

        # ignore "O" predictions and whitespace preds
        if label_pred == "O" or token_id == -1:
            continue
            
        pair = (doc, token_id)

        if pair in pairs:
            continue
            
        document.append(doc)
        token.append(token_id)
        label.append(label_pred)
        token_str.append(tokens[token_id])
        pairs.add(pair)

NameError: name 'preds_final' is not defined

This next part is creating the submission file for Kaggle

In [12]:
df_result = pd.DataFrame({
    "document": document,
    "token": token,
    "label": label,
    "token_str": token_str
})
df_result["row_id"] = list(range(len(df_result)))
display(df_result.head(100))

,document,token,label,token_str,row_id


In [13]:
df_result = df_result[['row_id', 'document', 'token', 'label']]
display(df_result.head(100))

,row_id,document,token,label
